# This is for original code I have removed from the project but I might still need later

# Heston simulation

In [ ]:
def paths(self, S: float, T: float, sigma: float, r: float, v0: float, kappa: float, theta: float, rho: float, M: int, I: int):

    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)

    dt = T / M

    ran_num = np.random.standard_normal((2, M + 1, I))

    S_paths = np.zeros_like(ran_num[0])
    v = np.zeros_like(ran_num[0])

    S_paths[0] = S
    v[0] = v0

    for t in range(1, M + 1):
        ran = np.dot(cho_mat, ran_num[:, t, :])

        v[t] = (v[t - 1] +
                kappa * (theta - np.maximum(v[t - 1], 0)) * dt +
                sigma * np.sqrt(np.maximum(v[t - 1], 0)) *
                np.sqrt(dt) * ran[1])
        
        S_paths[t] = S_paths[t - 1] * np.exp((r - 0.5 * v[t]) * dt +
                                np.sqrt(v[t]) * ran[0] * np.sqrt(dt))
        
    return S_paths

    def calculate(self, market_price: float, S: int, K: int, T: float, r: float = .05, initial_guess: float = .2, otype: str = "call", q = .02, **kwargs):

        interval: list[int] = [i/100000 for i in range(0, 200000)]
        
        first = 0
        last = len(interval) - 1
        while first <= last:
            mid = (first + last) // 2
            x = interval[mid]
            #print(f"iv {x}")

            if otype == 'call':
                model_price = self.model.call(S, K, T, x, r, q, **kwargs)
            else:
                model_price = self.model.put(S, K, T, x, r, q, **kwargs)
            print(f"abs(market_price - model_price): {abs(market_price - model_price)}")
            if abs(market_price - model_price) < .001:
                return x
            else:
                if market_price < model_price:
                    last = mid - 1
                else:
                    first = mid + 1
        return False

        iv = initial_guess

        for _ in range(1000):

            if otype == 'call':
                model_price = self.model.call(S, K, T, iv, r, q, **kwargs)
            else:
                model_price = self.model.put(S, K, T, iv, r, q, **kwargs)
            
            if abs(market_price - model_price) < .01:
                return iv

            if market_price - model_price > 0:
                lower_vol = iv
            else:
                upper_vol = iv

            iv = (lower_vol + upper_vol) / 2
            print(f"iv {iv}")

        return False


##### From process option chain py file, these were originaly functions for creating our OptionNode but I replaced them for classes that get passed in a specific Node

In [ ]:
def create_call_option_node(ticker: str, date: str, expiration: str, strike: int):

    #call_chain = SingleUpload().upload_chain(ticker, date)[0]
    chain = pu.cache_chain(ticker, date)
    call_chain = chain[0]
    chain_at_exp = call_chain[expiration]
    
    index: int = np.abs(chain_at_exp['Strike'] - strike).argmin()

    strike: int = chain_at_exp['Strike'][index]
    price: float = chain_at_exp['Price'][index]
    bid: float = chain_at_exp['Bid'][index]
    ask: float = chain_at_exp['Ask'][index]
    iv: float = chain_at_exp['Market Implied Volatility'][index]
    vol: int = chain_at_exp['Volume'][index]
    oi: int = chain_at_exp['Open Interest'][index]

    option_node: OptionNode = OptionNode(strike, price, bid, ask, iv, expiration, vol, oi, 'call')

    return option_node

def create_put_option_node(ticker: str, date: str, expiration: str, strike: int):

    chain = pu.cache_chain(ticker, date)

    put_chain = chain[1]

    #put_chain = SingleUpload().upload_chain(ticker, date)[1]
    chain_at_exp = put_chain[expiration]

    index: int = np.abs(chain_at_exp['Strike'] - strike).argmin()

    strike: int = chain_at_exp['Strike'][index]
    price: float = chain_at_exp['Price'][index]
    bid: float = chain_at_exp['Bid'][index]
    ask: float = chain_at_exp['Ask'][index]
    iv: float = chain_at_exp['Market Implied Volatility'][index]
    vol: int = chain_at_exp['Volume'][index]
    oi: int = chain_at_exp['Open Interest'][index]

    option_node: OptionNode = OptionNode(strike, price, bid, ask, iv, expiration, vol, oi, 'put')

    return option_node

In [ ]:
def process_call_data(ticker: str, date: str) -> OptionGraph:

    """This function creates a data strucutre object representing a list of options for a single expiration across strike"""

    expirations = pu.call_chain_exp(ticker, date)

    option_graph: OptionGraph = OptionGraph()

    for expiration in expirations:

        strikes = pu.call_exp_strikes(ticker, date, expiration)

        for strike in strikes:
            
            option = create_call_option_node(ticker, date, expiration, strike)

            option_graph.add_option(option)

    return option_graph

def process_put_data(ticker: str, date: str) -> OptionGraph:

    """This function creates a data strucutre object representing a graph of options for a single expiration across strike"""

    expirations = pu.put_chain_exp(ticker, date)

    option_graph: OptionGraph = OptionGraph()

    for expiration in expirations:

        strikes = pu.put_exp_strikes(ticker, date, expiration)

        for strike in strikes:
            
            option = create_put_option_node(ticker, date, expiration, strike)

            option_graph.add_option(option)

    return option_graph

### From process_option_chain
##### OptionFactory, now current one uses @staticmethod

In [ ]:
class OptionFactory:

    option_nodes: dict = {
        "european" : EuropeanOptionNode,
        "american" : AmericanOptionNode
    }

    def create_option(self, ticker: str, strike: int, expiration: str, close_date: str = TODAY, option_style: str = "european", option_type: str = "call") -> OptionNode:

        option = self.option_nodes.get(option_style.lower())

        if not option:

            return ValueError(f"Unsupported option type {option_style}")
        
        if option_type.lower() == "call":

            return CreateOption(ticker, close_date, expiration, strike).create_call(option)
        
        if option_type.lower() == "put":

            return CreateOption(ticker, close_date, expiration, strike).create_put(option)
        
        else:

            return ValueError(f"Option type {option_type} is not defined")
        
    def create_option_graph(self, ticker: str, close_date: str = TODAY, option_style: str = "european", option_type: str = "call"):
        
        option = self.option_nodes.get(option_style.lower())

        if not option:

            return ValueError(f"Unsupported option type {option_style}")
        
        if option_type.lower() == "call":

            return CreateOptionGraph(ticker, close_date).create_call_graph(option)
        
        if option_type.lower() == "put":

            return CreateOptionGraph(ticker, close_date).create_put_graph(option)
        
        else:

            return ValueError(f"Option type {option_type} is not defined")

## From option_pricing module
### bsm_model and monte_carlo
##### the two classes were meant to be factory/adapter like patterns where we can pass in a OptionNode into but we will save these for a client interface module

In [ ]:
class BSMCalculator:

    _dict: dict = {"call" : BSMCall(),
                   "put" : BSMPut()}

    def calculate(self, S: float, option_node: OptionNode, bsm_formula: str, r = 0, q = 0):

        formula = self._dict[bsm_formula]
        K: float = option_node.get_strike()
        T: float = option_node.get_dte() / 252
        sigma: float = option_node.get_implied_volatility()

        return formula(S, K, T, sigma, r, q)
    
class MonteCarloCalculator:

    def calculate(self, S: float, option_node: OptionNode, monte_carlo_method, r = 0, M = 100, I = 1000):

        K: float = option_node.get_strike()
        T: float = option_node.get_dte() / 252
        sigma: float = option_node.get_implied_volatility()

        return monte_carlo_method(S, K, M, I, T, sigma, r)

## From greek module

### analytical_greeks.py 
##### We got rid of a adapter like object that is passed in a OptionNode and calculates a greek, we will reimplement this later

In [ ]:
class AnalyticalGreekCalculation:

    # This is our Adapter pattern

    def __init__(self, greek: AnalyticalGreek) -> None:
        self._greek: AnalyticalGreek = greek

    def calculate(self, option_node: OptionNode, spot: int, r: float = 0, q: float = 0) -> float:
        strike: int = option_node.get_strike()
        dte: int = option_node.get_dte()
        sigma: float = option_node.get_implied_volatility()
        otype: str = option_node.otype

        return self._greek.calculate(spot, strike, dte, sigma, r, q, otype)

## Original Sticky delta parameterization

In [ ]:
class StickyDelta:

    # In this class we have a function that takes strikes as the x_axis not deltas
    # We manually calculate atm iv in the parameterize method, lets try to decouple this in the future

    def parameterize(self, implied_volatilities: list[float], strikes: list[float], **kwargs) -> list[float]:

        if "S" in kwargs:
            S = kwargs["S"]
        else:
            return ValueError("Keyword argument must have the parameter S")
        
        if "dte" in kwargs:
            dte = kwargs["dte"]

            if dte > 1:
                dte /= 365

        else:
            return ValueError("Keyword argument must have the parameter dte")

        logic = f"iv > .05 and iv < 2.5 and x <= {S}"

        atm_iv_filter: list[tuple(float, float)] = [(iv, x) for iv, x in zip(implied_volatilities, strikes) if eval(logic)]

        atm_iv: float = atm_iv_filter[-1][0]

        filtered_ivs_strikes: list[tuple(float, float)] = [(iv, np.log(x / S) / (atm_iv * np.sqrt(dte))) for iv, x in zip(implied_volatilities, strikes)]# if eval(func)]
        ivs, strikes = zip(*filtered_ivs_strikes)

        return ivs, strikes

## This function is used to calculate the list of a date range of historical stock prices, I opted out of using this function and decided to use the datetimes from the yfinance lib data

# 3/9/25 we got rid of module payoff_diagrams but it is below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from .positions import Position, PositionNode
from .position_calc import ChangeCalculator
from calc_engine.option_pricing import analytical_solutions as an

class PayoffDiagram:

    def __init__(self, payoff_diagram) -> None:
        self.payoff_diagram = payoff_diagram

    def plot(self, title: bool = False, xlabel: bool = False, ylabel: bool = False):

        y = self.payoff_diagram.get_payoff()
        x = self.payoff_diagram.get_change_variable()

        plt.figure()
        plt.plot(x, y)
        plt.show()

    def plot_exp(self, title: bool = False, xlabel: bool = False, ylabel: bool = False):

        y = self.payoff_diagram.get_payoff()
        y_exp = self.payoff_diagram.get_payoff_exp()
        x = self.payoff_diagram.get_change_variable()

        print(f"y expiration: {y_exp}\n")
        print(f"y: {y}\n")

        plt.figure()
        plt.plot(x, y, 'r')
        plt.plot(x, y_exp, 'b')
        plt.show()

    def plot_expiration(self):
        pass

class PayoffSpot:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp

    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("spot", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

        #payoff_diagram = PayoffDiagram().plot(change_variable, payoff)

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "spot", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "spot", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp

        #payoff_diagram = PayoffDiagram().plot(change_variable, position_payoff)

class PayoffSigma:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp
    
    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("sigma", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "sigma", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "sigma", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp

class PayoffTime:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp
    
    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("time", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "time", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "time", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp

## Below we got rid of position_calc.py from position_analysis on 5/13/25, it was not used anywhere

In [ ]:
import numpy as np

from .positions import Position, PositionNode
from calc_engine.option_pricing import analytical_solutions as an

""""
Below we have functions for calcualting the BSM price wrt a variable change represented as a np.array (spot, sigma, time, rate)
"""

def spot_change_calc(spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: 
                    float = 0, otype: str = "call", exposure: str = "long"):
    
    price_change = an.bsm_call(spot, strike, expiration, sigma, r, q) if otype == "call" else an.bsm_put(spot, strike, expiration, sigma, r, q)

    return price_change

def sigma_change_calc(spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: 
                    float = 0, otype: str = "call"):
    
    print(f"sigma: {sigma}\n")
    
    price_change = an.bsm_call(spot, strike, expiration, sigma, r, q) if otype == "call" else an.bsm_put(spot, strike, expiration, sigma, r, q)
    print(f"price_change: {price_change}\n")
    print(f"____________________________\n")
    return price_change

def rate_change_calc(spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: 
                    float = 0, otype: str = "call"):
    
    price_change = an.bsm_call(spot, strike, expiration, sigma, r, q) if otype == "call" else an.bsm_put(spot, strike, expiration, sigma, r, q)

    return price_change

def time_change_calc(spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: 
                    float = 0, otype: str = "call"):
    
    price_change = an.bsm_call(spot, strike, expiration, sigma, r, q) if otype == "call" else an.bsm_put(spot, strike, expiration, sigma, r, q)

    return price_change

def premium_calc(spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: 
                    float = 0, otype: str = "call", exposure: str = "long"):
    
    premium = an.bsm_call(spot, strike, expiration, sigma, r, q) if otype == "call" else an.bsm_put(spot, strike, expiration, sigma, r, q)

    if exposure != "long" and exposure != "short":
        exposure = input("You must enter either long or short for exposure\n would you like to go [long] or [short]?\n")

    if exposure == "long": return -premium

    else:
        return premium

class ChangeCalculator:
    
    func_dict: dict = {"spot": spot_change_calc, "sigma": sigma_change_calc, "time": time_change_calc, "rate": rate_change_calc}

    def calculate_change(self, change_variable_name: str, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, 
                  r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):
        
        params_str = ["spot", "sigma", "time", "r", "q"]
        params = [spot, sigma, expiration, r, q]

        if change_variable_name in params_str:
            
            var_index = params_str.index(change_variable_name)
            params[var_index] = change_variable

        else:
            raise ValueError("Incorrect change variable name parameter")
        
        if exposure != "long" and exposure != "short":
            exposure = input("You must enter either long or short for exposure\n would you like to go [long] or [short]?\n")

        prices = self.func_dict[change_variable_name](params[0], strike, params[2], params[1], params[3], params[4], otype)

        premium = premium_calc(spot, strike, expiration, sigma, r, q, otype, exposure)

        if exposure == "long": 
            print(f"in long, exposure = {exposure}")
            return prices + premium

        else:
            print(f"in short, exposure = {exposure}")
            return -prices + premium
    
    def calculate_position_node(self, position_node: PositionNode, change_variable_name: str, change_variable: np.ndarray | list):

        spot: float = position_node.get_spot()
        exposure: str = position_node.get_exposure()

        if isinstance(position_node._stock_position, (list, np.ndarray)):
            return position_node._stock_position - spot if exposure == "long" else -position_node._stock_position + spot

        strike: int = position_node.get_strike()
        expiration: float = position_node.get_expiration()
        sigma: float = position_node.get_sigma()
        r: float = position_node.get_r()
        q: float = position_node.get_q()
        otype: str = position_node.get_otype()

        position_change = self.calculate_change(change_variable_name, change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        #premium = premium_calc(spot, strike, expiration, sigma, r, q, otype, exposure)

        position_payoff = position_change #+ premium
        
        #print(f"premium {premium}\n")

        return position_payoff
    
    def calculate_position_node_expiration(self, position_node: PositionNode, change_variable_name: str, change_variable: np.ndarray | list):

        print(position_node)

        spot: float = position_node.get_spot()
        exposure: str = position_node.get_exposure()

        if isinstance(position_node._stock_position, (list, np.ndarray)):
            return position_node._stock_position - spot if exposure == "long" else -position_node._stock_position + spot

        strike: int = position_node.get_strike()
        expiration: float = position_node.get_expiration()
        sigma: float = position_node.get_sigma()
        r: float = position_node.get_r()
        q: float = position_node.get_q()
        otype: str = position_node.get_otype()

        position_change = self.calculate_change(change_variable_name, change_variable, spot, strike, .0001, sigma, r, q, otype, exposure)

        premium = premium_calc(spot, strike, expiration, sigma, r, q, otype, exposure)

        position_payoff = position_change + premium
        
        print(f"premium {premium}\n")
        print(f"position_change[0] {position_change[0]}\n")
        print(f"position_change[-1] {position_change[-1]}\n")

        return position_payoff
    
    def calculate_position(self, positions: Position, change_variable_name: str, change_variable: np.ndarray | list):

        prices = [self.calculate_position_node(position, change_variable_name, change_variable) for position in positions]

        payoff = prices[0]

        print(f"prices[{0}]: {prices[0]}\n")

        for p in range(1, len(prices)):
            print(p)
            print(f"prices[{p}]: {prices[p]}\n")
            payoff += prices[p]


        return payoff
    
        
    def calculate_position_expiration(self, positions: Position, change_variable_name: str, change_variable: np.ndarray | list):

        prices = [self.calculate_position_node_expiration(position, change_variable_name, change_variable) for position in positions]

        payoff = prices[0]

        print(f"prices[{0}]: {prices[0]}\n")

        for p in range(1, len(prices)):
            print(p)
            print(f"prices[{p}]: {prices[p]}\n")
            payoff += prices[p]


        return payoff


## Below we have two .py files, back_test from position_analysis and back_test from visualizations. Both of these will be used later but now are messy and need to be re done. back_test.py from position_analysis has very useful functions so we will probably come back here later to grab them or remake the ones below, the most important plot a certain option over a time frame like a historical time series for the option, we should use this in collabortation with postions.py in position_analysis to create historical back tests of option positions. Maybe use some pattern that stands in between back_test.py and positions.py and links them together like adapter or bridge.
## These were removes on 5/13/25

#### back_test.py - from position_analysis

In [ ]:
from .positions import Position
from data.option_data import OptionFactory
import utils

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np

def historical_option_prices(ticker: str, K: int, start_date: str, end_date: str, expiration: str, option_style: str = "european", option_type: str = "call") -> list[float]:

    close_dates: list[str] = utils.option_date_range(start_date, end_date)

    meta_option_price_list: list[int] = []

    for close_date in close_dates:
        option_price: float = OptionFactory().create_option(ticker, K, expiration, close_date, option_style, option_type).get_price()
        meta_option_price_list.append(option_price)

    return meta_option_price_list, close_dates

def position_back_test(ticker: str, start_date: str, end_date: str, position: Position):

    date_list: list = []
    position_price: float = 0
    print(f"position: {position}\n")

    for pos in position:

        strike = pos.get_strike()
        expiration = pos.get_expiration()
        option_type = pos.get_otype()
        exposure = pos.get_exposure()
        #print(f"pos {pos}\n")

        option_price, date = historical_option_prices(ticker, strike, start_date, end_date, expiration, option_type=option_type)
        op_price = np.array(option_price)

        if exposure == "long":
            position_price += op_price
        elif exposure == "short":
            position_price -= op_price
        else:
            raise ValueError("exposure must be long or short")

        date_list.append(date)

    return position_price, date_list

def plot_back_test(ticker: str, start_date: str, end_date: str, position: Position):
    backt = position_back_test(ticker, start_date, end_date, position)
    print(f"backt in back_test.py: {backt}")
    fig, ax = plt.subplots()
    ax.plot(backt[1][0], backt[0])
    ax.xaxis.set_major_locator(MaxNLocator(nbins=12))
    plt.xticks(rotation=45)
    plt.title(f"{ticker} ")
    plt.show()
    #plt.figure()
    #plt.plot(backt[1][0], backt[0])
    #plt.xticks(rotation=45) 
    #plt.show()


def option_back_test_date_check(ticker: str, start_date: str, end_date: str, option_exp_date: str):

    option_factory = OptionFactory()

    close_date_range = utils.option_date_range(start_date, end_date)

    option_graph_list = []

    exp_list = []

    for date in close_date_range:
        print(date)

        option_graph = option_factory.create_option_graph(ticker, date)
        option_graph_list.append(option_graph)

    print(f"option graph list: {option_graph_list}\n")

    for i, option in enumerate(option_graph_list):
        exp = option.get_expirations()
        #print(f"exp: {exp}\n")
        print(f"exp: {exp}\n")

        if option_exp_date not in exp:
            print(f"exp: {exp}\n")
            print(f"in break \n")
            print(f"option node: {option}")
            #raise ValueError("The Expiration date you picked is not available across the back test date range")
            continue
        
        else:
            #print(f"not in break \n")
            #print(f"option node: {option.nodes}\n")
            exp_list.append(exp)

    return exp_list

def main():

    option_back_test_date_check("SPY", "2024-02-10", "2024-04-19", "2024-04-19")


    return None

#### back_test.py from visualizations

In [ ]:
class BackTest:
    """
    We need to delete this and turn it into a seperate .py file
    """

    def __init__(self):
        if "ticker" not in st.session_state:
            st.session_state["ticker"] = ""
        if "strike" not in st.session_state:
            st.session_state["strike"] = ""
        if "expiration" not in st.session_state:
            st.session_state["expiration"] = ""
        if "option_start_date" not in st.session_state:
            st.session_state["option_start_date"] = ""
        if "option_end_date" not in st.session_state:
            st.session_state["option_end_date"] = ""

    def create_display(self):
        st.session_state["ticker"] = st.text_input("Enter a Ticker", st.session_state["ticker"])
        st.session_state["strike"] = st.text_input("Enter a Strike", st.session_state["strike"])
        st.session_state["expiration"] = st.text_input("Enter the Option Expiration ('YYYY-MM-DD')", st.session_state["expiration"])
        st.session_state["option_start_date"] = st.text_input("Enter the Option Start Date ('YYYY-MM-DD')", st.session_state["option_start_date"])
        st.session_state["option_end_date"] = st.text_input("Enter the Option End Date ('YYYY-MM-DD')", st.session_state["option_end_date"])

    def plot_display(self):
        option_prices, dates = bt.historical_option_price(str(st.session_state["ticker"]), int(st.session_state["strike"]), str(st.session_state["option_start_date"]), str(st.session_state["option_end_date"]), str(st.session_state["expiration"]))

        fig, ax = plt.subplots()
        ax.plot(dates, option_prices)

        ax.set_xlabel("Strike Prices")
        ax.set_ylabel("Implied Volatilities")
        ax.set_title(f"Performance")

        plt.xticks(rotation=45)

        return st.pyplot(fig)
    
def main():
    pass

## Below is from volatility_processor.py I did not delete the file I just removed the code because I want to implement it in a better less coupled way. This code is highly coupled with qpp/volatility_surface.py which is used for visualizing the surface.
### Removed on 5/13/25

In [ ]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial

from data.option_data.process_option_chain import OptionFactory
from calc_engine.volatility import iv_calc
from calc_engine.volatility import local_volatility as lv
from calc_engine.option_pricing import analytical_solutions as an

def market_price_generator(ticker: str, close_date: str, strikes: int, expirations=None, option_type: str = 'call'):
    """
    Generator that yields OptionNode objects based on specified expirations and strikes.

    :param option_type: Type of option ('call' or 'put')
    :param expirations: List of expiration dates to process
    :param strikes: List of strikes to process
    :yield: OptionNode object
    """
    option_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)
    
    if expirations is None:
        expirations = list(option_graph.get_expirations())

    atm_option = option_graph.get_atm_option(expirations[1], close_date)
    atm_strike = atm_option.get_strike()
    atm_iv = atm_option.get_implied_volatility()

    for exp in expirations:
        price_list = []
        new_strikes = []
        for strike in strikes:

            if (exp, strike) not in option_graph.nodes:
                continue

            option = option_graph.nodes[(exp, strike)]

            market_price = option.get_price()
            dte = option.get_dte()
            new_strikes.append(strike)
            price_list.append(market_price)

        yield (price_list, dte, new_strikes, atm_strike, atm_iv)

def market_iv_generator(ticker: str, close_date: str, strikes: int, expirations=None, option_type: str = 'call'):
    """
    Generator that yields OptionNode objects based on specified expirations and strikes.

    :param option_type: Type of option ('call' or 'put')
    :param expirations: List of expiration dates to process
    :param strikes: List of strikes to process
    :yield: OptionNode object
    """
    option_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)
    
    if expirations is None:
        expirations = list(option_graph.get_expirations())

    atm_option = option_graph.get_atm_option(expirations[1], close_date)
    atm_strike = atm_option.get_strike()

    for exp in expirations:
        iv_list = []
        new_strikes = []
        for strike in strikes:

            if (exp, strike) not in option_graph.nodes:
                continue

            option = option_graph.nodes[(exp, strike)]

            market_iv = option.get_implied_volatility()
            market_price = option.get_price()
            dte = option.get_dte()
            new_strikes.append(strike)

            iv_list.append(market_iv)

        yield (iv_list, dte, new_strikes, atm_strike)

def bsm_iv_generator(ticker: str, close_date: str, strikes: int, expirations=None, model = an.BlackScholesMertonAnalytical(), option_type: str = 'call'):
    """
    Generator that yields OptionNode objects based on specified expirations and strikes.

    :param option_type: Type of option ('call' or 'put')
    :param expirations: List of expiration dates to process
    :param strikes: List of strikes to process
    :yield: OptionNode object
    """
    option_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)
    iv_calc_obj = iv_calc.ImpliedVolatility()
    
    if expirations is None:
        expirations = list(option_graph.get_expirations())

    atm_option = option_graph.get_atm_option(expirations[1], close_date)
    atm_strike = atm_option.get_strike()

    for exp in expirations:
        iv_list = []
        new_strikes = []
        for strike in strikes:

            if (exp, strike) not in option_graph.nodes:
                continue

            option = option_graph.nodes[(exp, strike)]

            market_iv = option.get_implied_volatility()
            market_price = option.get_price()
            dte = option.get_dte()
            new_strikes.append(strike)

            bs_iv = iv_calc_obj.newtons_method(market_price, atm_strike, strike, dte, 0, otype=option_type)

            if bs_iv == 0 or bs_iv is None:
                iv_list.append(market_iv * 100)
            else:
                iv_list.append(bs_iv)
                    
        yield (iv_list, dte, new_strikes)

def local_volatility_generator(ticker: str, close_date: str, strikes: int, expirations=None, option_type: str = 'call'):
    option_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)
    lv_calc_obj = lv.LocalVolatility()
    
    if expirations is None:
        expirations = list(option_graph.get_expirations())

    atm_option = option_graph.get_atm_option(expirations[1])
    atm_strike = atm_option.get_strike()

    for exp in expirations:
        local_vol_list = []
        new_strikes = []

        for strike in strikes:

            if (exp, strike) not in option_graph.nodes:
                continue

            option = option_graph.nodes[(exp, strike)]

            market_iv = option.get_implied_volatility()
            market_price = option.get_price()
            dte = option.get_dte()
            new_strikes.append(strike)

            local_vol = lv_calc_obj.dupire_finite_difference(atm_strike, strike, dte, market_iv, otype=option_type)

            if local_vol == 0 or local_vol == None:
                local_vol_list.append(market_iv * 100)
            else:
                local_vol_list.append(local_vol * 100)
        
        yield (local_vol_list, dte, new_strikes)

def local_volatility_surface_data_processor(ticker: str, close_date: str, expirations: list[str], strikes: list[float], option_type: str):
    local_vol_obj = lv.LocalVolatility()
    o_graph = OptionFactory().create_option_graph(ticker, close_date, option_type = option_type)
    dtes = []
    price_surface = []

    print(f"o graph {o_graph}")

    for exp in expirations:
        skew = o_graph.get_skew(exp)
        dte = skew.get_dte()
        dtes.append(dte)
        price_list = []
        new_strikes = []

        for option in skew:
            if option.get_strike() in strikes:
                price_list.append(option.get_price())
                new_strikes.append(option.get_strike())

        print(f"strikes {len(strikes)}")
        print(f"price list {len(price_list)}\n")
        new_strikes = np.array(new_strikes)
        p = Polynomial.fit(np.array(new_strikes), np.array(price_list), deg=3)
        x_poly = np.linspace(new_strikes.min(), new_strikes.max(), 100)
        y_poly = p(x_poly)
        
        price_surface.append(y_poly)
    
    price_surface = np.array(price_surface)
    dtes = np.array(dtes) / 252

    loc_vol_surface = local_vol_obj.dupire_price_surface(x_poly, dtes, price_surface)

    return loc_vol_surface, x_poly, dtes

def sabr_generator(ticker: str, close_date: str, strikes: int, expirations=None, model = an.BlackScholesMertonAnalytical(), option_type: str = 'call'):
    option_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)
    iv_calc_obj = iv_calc.ImpliedVolatility(model=model)
    
    if expirations is None:
        expirations = list(option_graph.get_expirations())

    atm_option = option_graph.get_atm_option(expirations[1], close_date)
    atm_strike = atm_option.get_strike()

    for exp in expirations:
        iv_list = []
        new_strikes = []
        for strike in strikes:

            if (exp, strike) not in option_graph.nodes:
                continue

            option = option_graph.nodes[(exp, strike)]

            market_iv = option.get_implied_volatility()
            market_price = option.get_price()
            dte = option.get_dte()
            new_strikes.append(strike)

            bs_iv = iv_calc_obj.newtons_method(market_price, atm_strike, strike, dte, 0, otype=option_type)

            if bs_iv == 0 or bs_iv is None:
                iv_list.append(market_iv * 100)
            else:
                iv_list.append(bs_iv)
                    
        yield (iv_list, dte, new_strikes)

def price_surface_heston(ticker: str, close_date: str, strikes: list[int], expirations: list[str] = None, option_type: str = 'call'):
    o_graph = OptionFactory().create_option_graph(ticker, close_date, option_type=option_type)

    if expirations is None:
        expirations = list(o_graph.get_expirations())

    dte_list = []
    price_surf = []
    strike_data_list = strikes

    #atm_option = o_graph.get_atm_option(expirations[1], close_date)
    #atm_strike = atm_option.get_strike()
    #tm_iv = atm_option.get_implied_volatility()

    for exp in expirations:
        price_list = []
        indices_to_remove = []  # Collect indices to remove
        skew = o_graph.get_skew(exp)
        dte = skew.get_dte()
        dte_list.append(dte)


        # Identify indices to remove
        for idx, strike in enumerate(strike_data_list):
            if (exp, strike) not in o_graph.nodes:
                indices_to_remove.append(idx)
            else:
                option = o_graph.nodes[(exp, strike)]
                price_list.append(option.get_price())

        # Remove items in reverse order to avoid index shifting
        for idx in sorted(indices_to_remove, reverse=True):
            for p_list in price_surf:
                p_list.pop(idx)
            strike_data_list.pop(idx)

        price_surf.append(price_list)

    return price_surf, dte_list, strike_data_list, atm_strike, atm_iv

In [ ]:
    def call(self, S0, K, T, sigma=0.2, alpha=0.8, beta=0.9, rho=-0.5, r=0.04, q=0, I=1000, M=1000):

        paths = self.paths(S0, T, sigma, alpha, beta, rho, r, I, M)

        if isinstance(K, (int, float)):
            K = [K]

        ST = paths[-1][:, np.newaxis]
        K = np.array(K)[np.newaxis, :]      

        payoffs = np.maximum(ST - K, 0)     
        prices = np.exp(-r * T) * np.mean(payoffs, axis=0) 
        return prices

    def put(self, S0, K, T, sigma=0.2, alpha=0.8, beta=0.9, rho=-0.5, r=0.04, q=0, I=1000, M=1000):

        paths = self.paths(S0, T, sigma, alpha, beta, rho, r, I, M)

        if isinstance(K, (int, float)):
            K = [K]

        ST = paths[-1][:, np.newaxis]
        K = np.array(K)[np.newaxis, :]      

        payoffs = np.maximum(K - ST, 0)     
        prices = np.exp(-r * T) * np.mean(payoffs, axis=0) 
        return prices

## Below is my origianl homescreen for the first py_op visualizer, I am remaking I deleted this on 5/18/25

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import sys
import os

from calc_engine.option_pricing import analytical_solutions as an
from . import gamma_exposure as ge
from . import realized_volatility as rv
from . import risk_measures as rm
from . import back_test as bt

def main_button():
    # Check if `selected_option` exists in session state
    if "selected_option" not in st.session_state:
        st.session_state.selected_option = None

    # Create horizontal buttons using `st.columns()`
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.markdown("Volatility")
        if st.button("Volatility Surface"):
            st.session_state.selected_option = "Volatility Surface"

        if st.button("Realized Volatility"):
            st.session_state.selected_option = "Realized Volatility"

    with col2:
        st.markdown("Position Analysis")
        if st.button("Back Testing"):
            st.session_state.selected_option = "Back Testing"

        if st.button("Risk Measures"):
            st.session_state.selected_option = "Risk Measures"

    with col3:
        st.markdown("Analysis Tools")
        if st.button("Gamma Exposure"):
            st.session_state.selected_option = "Gamma Exposure"

    # Display the current selected option
    st.write(f"Selected Option: {st.session_state.selected_option}")

    return st.session_state.selected_option
        
SCREEN_FACTORY = {'Back Testing' : bt.main,
                  'Gamma Exposure' : ge.main,
                  "Realized Volatility" : rv.main,
                  "Risk Measures" : rm.main}

def main():

    st.markdown("# Option Research Platform")

    st.session_state['current_screen'] = main_button()

    #try:
    
    display_object = SCREEN_FACTORY[str(st.session_state["current_screen"])]

    display_object()

        #display_object.create_display()

        #display_object.plot_display()
    
    #except:
        #st.write("None")

#main()

In [3]:
import numpy as np
l = [[1,2, 3], [2, 3]]
l1 = [[2, 3, 4], [2, 3, 4]]
np.array(l) + np.array(l1)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
    def manual_optimization(self, S: float, strikes: list[float], prices: list[list[float]], T: list[float], v0: float, theta: float, r: float = 0, q: float = 0, otype = 'call'):
        """
        This function is not done but we saved the code here for future updating
        We probably shouldnt run this because it will take like an hour and nobody got time for that just use scipy
        Turn theta into sigma(atm) for specific T, or use k / dte where k =2.75
        """
        kappa_list = [2, 2.5, 3.5, 5]
        rho_list = [-1, -.8, -.6, -.5]
        sigma_list = [.2, .3, .5, .8]
        sse_hest = []
        for k in kappa_list:
            for rh in rho_list:
                for sig in sigma_list:

                    # implement feller condition
                    if 2*k*theta - sig**2 < 0:
                        continue

                    erf_sum_heston = self.error_function(S, strikes, prices, np.array(T) / 252, self.model, self.calculation_type, v0 = v0, kappa = k, theta = theta, rho = rh, sigma = sig)
                    sse_hest.append((erf_sum_heston, k, rh, sig))

        min_tuple = min(sse_hest, key=lambda x: x[0])

        return min_tuple


    def manual_optimizer(self, S: float, strikes: list[float], ivs: list[float], T: float, beta: float=.99):
        alpha_list = np.linspace(.01, 5, 30) #[.1, .2, .4, .6, .8] 
        rho_list = np.linspace(-.99, .99, 30) #[-.5, -.2, .2, .5, .8]
        sigma_list = np.linspace(.01, .8, 30) #[.1, .25, .4, .5, .65]

        sse_list = []
        for i in range(len(alpha_list)):
            for j in range(len(sigma_list)):
                for k in range(len(rho_list)):
                    sse = self.error_function(S, strikes, ivs, T, self.model, self.calculation_type, sigma_0=sigma_list[j], alpha=alpha_list[i], beta=beta, rho=rho_list[k])
                    sse_list.append((sse, sigma_list[j], alpha_list[i], rho_list[k]))

        min_params = min(sse_list)

        return [min_params[1], min_params[2], min_params[3]]

## Below is the realized volatility cone code, we will remake this so it is inline with software design principles
### RIP 5/27/25

In [ ]:
import matplotlib.pyplot as plt

from ..time_series.historical_price_volatility import HistoricalVolatilityStrategy

def realized_volatility_cone(historical_volatility_strategy: HistoricalVolatilityStrategy, ticker: str, start: str, end: str = None, freq: str = "D") -> (list[float], list[float], list[float]):

    vol_data = [historical_volatility_strategy.calculate(ticker, start, end, realized_volatility_period=i, freq=freq) for i in [20, 40, 60, 120]]

    max_vol = [max(vol) for vol in vol_data]
    avg_vol = [sum(vol) / len(vol) for vol in vol_data]
    min_vol = [min(vol) for vol in vol_data]

    return (max_vol, avg_vol, min_vol)

def plot_volatility_cone(historical_volatility_strategy: HistoricalVolatilityStrategy, ticker: str, start: str, end: str = None, freq: str = "D") -> (list[float], list[float], list[float]):
    vol_cone_data = realized_volatility_cone(historical_volatility_strategy, ticker, start, end=end, freq=freq)

    x = ["20 Days", "40 Days", "60 Days", "120 Days"]

    plt.plot(x, vol_cone_data[0], label='Max')
    plt.plot(x, vol_cone_data[1], label='Average')
    plt.plot(x, vol_cone_data[2], label='Min')

    plt.legend()
    plt.title(f'{historical_volatility_strategy} Realized Volatility Cone')
    plt.xlabel('Length of Realized Volatility Period')
    plt.ylabel('Realized Volatility')

    plt.tight_layout()

    plt.show()



# Old get_prices method from process_price_data.py

In [ ]:
def get_close_prices_old(ticker: str, start: str, end: str = None, freq: str = "D") -> list[float]:

    prices1 = get_from_cache(ticker, start, end, freq)["Close"].dropna()

    #prices2 = get_prices(ticker, start, end, freq)

    return prices1#, filtered_dates

# GVV using ols
## Deleted on 6/2/25

In [ ]:
def gvv_error_func(S, K, dte, b1, b2, b3, iv, r=0, q=0, otype='call'):

    k = np.log(K / S)
    z_p = k + .5 * iv **2 * dte
    z_m = k - .5 * iv **2 * dte

    dollar_gammas = AnalyticalGamma().calculate(S, K, dte, iv, r=r, q=q, otype=otype)*S**2
    dollar_thetas = -.5* iv **2 * dollar_gammas
    dollar_vannas = z_p * dollar_gammas
    dollar_volgas = z_p*z_m * dollar_gammas

    lhs = -dollar_thetas
    rhs = b1 * dollar_gammas + b2 * dollar_vannas + b3 * dollar_volgas

    return lhs - rhs

class GVV:

    def calculate(self, S, strikes, dte, ivs) -> None:
        k = np.log(strikes / S)
        z_p = k + .5 * ivs**2 * dte
        z_m = k - .5 * ivs**2 * dte

        dollar_gammas = AnalyticalGamma().calculate(S, strikes, dte, ivs)*S**2
        dollar_thetas = -.5*ivs**2 * dollar_gammas
        dollar_vannas = z_p * dollar_gammas
        dollar_volgas = z_p*z_m * dollar_gammas
        
        X = np.column_stack((dollar_gammas, dollar_vannas, dollar_volgas))
        y = -dollar_thetas

        model = sm.OLS(y, X)
        results = model.fit()

        b1, b2, b3 = results.params
        return b1, b2, b3
    
    def implied_vol(self, S, strikes, dte, ivs):
        b1, _, _ = self.calculate(S, strikes, dte, ivs)
        return 2*b1
    
    def implied_spot_vol_corr(self, S, strikes, dte, ivs):
        b1, b2, b3 = self.calculate(S, strikes, dte, ivs)
        v = 2*b1
        omega = np.sqrt(2*b3)
        return b2/(omega*np.sqrt(v))
    
    def implied_vol_vol(self, S, strikes, dte, ivs):
        _, _, b3 = self.calculate(S, strikes, dte, ivs)
        return np.sqrt(2*b3)
    
    def skew(self, S, strikes, dte, ivs):
        ivs_root = []
        b1, b2, b3 = self.calculate(S, strikes, dte, ivs)
        
        for K in strikes:
            f = lambda iv: gvv_error_func(S=S, K=K, dte=dte, b1=b1, b2=b2, b3=b3, iv=iv[0])
            sol = root(f, x0=[0.2], method='hybr')
            ivs_root.append(sol.x[0])
            #sol = root_scalar(lambda iv: gvv_error_func(S=S, K=K, dte=dte, b1=b1, b2=b2, b3=b3, iv=iv), bracket=[0.01, 1.0], method='brentq')
            #ivs_root.append(sol.root)

        return ivs_root

In [ ]:
import numpy as np
from scipy.optimize import least_squares

from ..greeks.analytical_greeks import AnalyticalGamma
from utils.model_utils import GVVUtils

class GVV(GVVUtils):

    def __init__(self, model_gamma = AnalyticalGamma()) -> None:
        super().__init__(model_gamma)

    def calculate(self, S, strikes, dte, ivs):
        k = np.log(strikes / S)
        z_p = k + 0.5 * ivs**2 * dte
        z_m = k - 0.5 * ivs**2 * dte

        gamma = self.model_gamma.calculate(S, strikes, dte, ivs) * S**2
        theta = -0.5 * ivs**2 * gamma
        vanna = z_p * gamma
        volga = z_p * z_m * gamma

        X = np.column_stack((gamma, vanna, volga))
        y = -theta

        # Solve OLS for current iv guess
        coeffs, *_ = np.linalg.lstsq(X, y, rcond=None)
        b1, b2, b3 = coeffs
        return b1, b2, b3

    def implied_vol(self, S, strikes, dte, ivs):
        b1, _, _ = self.calculate(S, strikes, dte, ivs)
        return 2*b1
    
    def implied_spot_vol_corr(self, S, strikes, dte, ivs):
        b1, b2, b3 = self.calculate(S, strikes, dte, ivs)
        v = 2*b1
        omega = np.sqrt(2*b3)
        return b2/(omega*np.sqrt(v))
    
    def implied_vol_vol(self, S, strikes, dte, ivs):
        _, _, b3 = self.calculate(S, strikes, dte, ivs)
        return np.sqrt(2*b3)

    def skew(self, S, strikes, dte, ivs_init, method = 'least squares'):

        if method == 'least squares':
            res = least_squares(
                self.gvv_error_func_least_squares, ivs_init,
                args=(S, strikes, dte),
                bounds=(0.01, 2.0),
                xtol=1e-8, ftol=1e-8
            )
            return res.x
        
        if method == 'root finder':
            b1, b2, b3 = self.calculate(S, strikes, dte, ivs_init)
            ivs_rooted = []
            for K in strikes:
                iv = self.get_iv_binary_search(S, K, dte, b1, b2, b3)
                ivs_rooted.append(iv)
            return np.array(ivs_rooted)

## Sum of Squares Puts and Call
### This was in error_functions from calibration, this was used in rBergomi, we will create a more robust way of optimizing
### 6/7/2025

In [ ]:

def sum_of_squares_puts_and_calls(S: float, strikes: np.ndarray, market_prices: np.ndarray, T: float, model, **kwargs) -> float:
    """
    Params:
    float S: Spot price of asset.
    list[int] strikes: List of strike prices, we might turn this into an np.array.
    float T: time to expiration, should be of form days/252.
    list[int] prices: Market option prices, we need might turn this into a np.array.
    model: Our specific option pricing model, must have a call or put method.

    Notes: 
    This error function is used when our price data is a list composed of both otm calls and puts.
    This error function does not rely on our data processor module but we make another error function that does rely on it.
    """
    assert len(strikes) == len(market_prices), "Strikes and market prices must be the same length"

    put_indices = np.where(strikes <= S)[0]
    call_indices = np.where(strikes > S)[0]
    print(f"market prices: {market_prices}")
    print(f"call indices: {call_indices}")
    market_call = market_prices[call_indices]
    market_put = market_prices[put_indices]

    call_strikes = strikes[call_indices]
    put_strikes = strikes[put_indices]

    model_call = model.call(S, call_strikes, T, **kwargs)
    sum_call = np.sum((market_call - model_call)**2)
    
    model_put = model.put(S, put_strikes, T, **kwargs)
    sum_put = np.sum((market_put - model_put)**2)

    return sum_call + sum_put
